# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J. Syed  ->  J. Syed  |  ['J. Syed']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 72 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.01725


extracting tarball to tmp_2310.01725...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.02077


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.01725/draft_aastex.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Found 99 bibliographic references in tmp_2310.02077/main.bbl.
syntax error in line 12: unbalanced braces
Retrieving document from  https://arxiv.org/e-print/2310.02213


extracting tarball to tmp_2310.02213...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']


/tmp/ipykernel_2535/3009462391.py:49: LatexWarning: 2310.02213 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.02077-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.02077) | **Cold atomic gas identified by HI self-absorption. Cold atomic clouds  toward giant molecular filaments**  |
|| <mark>J. Syed</mark>, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2023-10-04*|
|*Comments*| *41 pages, 28 figures, accepted for publication in A&A*|
|**Abstract**| Stars form in the dense interiors of molecular clouds. The dynamics and physical properties of the atomic interstellar medium (ISM) set the conditions under which molecular clouds and eventually stars will form. It is, therefore, critical to investigate the relationship between the atomic and molecular gas phase to understand the global star formation process. Using the high angular resolution data from The HI/OH/Recombination line survey of the Milky Way (THOR), we aim to constrain the kinematic and physical properties of the cold atomic hydrogen gas phase toward the inner Galactic plane. HI self-absorption (HISA) has proven to be a viable method to detect cold atomic hydrogen clouds in the Galactic plane. With the help of a newly developed self-absorption extraction routine (astroSABER), we build upon previous case studies to identify HI self-absorption toward a sample of Giant Molecular Filaments (GMFs). We find the cold atomic gas to be spatially correlated with the molecular gas on a global scale. The column densities of the cold atomic gas traced by HISA are usually of the order of $10^{20}\rm\,cm^{-2}$ whereas those of molecular hydrogen traced by $\rm^{13}CO$ are at least an order of magnitude higher. The HISA column densities are attributed to a cold gas component that accounts for a fraction of $\sim$5% of the total atomic gas budget within the clouds. The HISA column density distributions show pronounced log-normal shapes that are broader than those traced by HI emission. The cold atomic gas is found to be moderately supersonic with Mach numbers of a $\sim$few. In contrast, highly supersonic dynamics drive the molecular gas within most filaments. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.01725-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.01725) | **Super-Earth LHS3844b is tidally locked**  |
|| X. Lyu, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2023-10-04*|
|*Comments*| *Submitted*|
|**Abstract**| Short period exoplanets on circular orbits are thought to be tidally locked into synchronous rotation. If tidally locked, these planets must possess permanent day- and nightsides, with extreme irradiation on the dayside and none on the nightside. However, so far the tidal locking hypothesis for exoplanets is supported by little to no empirical evidence. Previous work showed that the super-Earth LHS 3844b likely has no atmosphere, which makes it ideal for constraining the planet's rotation. Here we revisit the Spitzer phase curve of LHS 3844b with a thermal model of an atmosphere-less planet and analyze the impact of non-synchronous rotation, eccentricity, tidal dissipation, and surface composition. Based on the lack of observed strong tidal heating we rule out rapid non-synchronous rotation (including a Mercury-like 3:2 spin-orbit resonance) and constrain the planet's eccentricity to less than 0.001 (more circular than Io's orbit). In addition, LHS 3844b's phase curve implies that the planet either still experiences weak tidal heating via a small-but-nonzero eccentricity (requiring an undetected orbital companion), or that its surface has been darkened by space weathering; of these two scenarios we consider space weathering more likely. Our results thus support the hypothesis that short period rocky exoplanets are tidally locked, and further show that space weathering can significantly modify the surfaces of atmosphere-less exoplanets. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.02213-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.02213) | **Turbulent Structure In Supernova Remnants G46.8-0.3 And G39.2-0.3 From  THOR Polarimetry**  |
|| R. Shanahan, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2023-10-04*|
|*Comments*| *Has been accepted by ApJ for publication. Figures 3 and 4 did do not render well when using an internet browser to view them, but when the pdf file is downloaded these figures look as they should*|
|**Abstract**| We present the continued analysis of polarization and Faraday rotation for the supernova remnants (SNRs) G46.8-0.3 and G39.2-0.3 in L-band (1-2 GHz) radio continuum in The HI/OH/Recombination line (THOR) survey. In this work, we present our investigation of Faraday depth fluctuations from angular scales comparable to the size of the SNRs down to scales less than our 16" beam (<~0.7 pc) from Faraday dispersion (sigma_phi). From THOR, we find median sigma_phi of 15.9 +/- 3.2 rad m^-2 for G46.8-0.3 and 17.6 +/- 1.6 rad m^-2 for G39.2-0.3. When comparing to polarization at 6cm, we find evidence for sigma_phi >~ 30 rad m^-2 in localized regions where we detect no polarization in THOR. We combine Faraday depth dispersion with the rotation measure (RM) structure function (SF) and find evidence for a break in the SF on scales less than the THOR beam. We estimate the RM SF of the foreground interstellar medium (ISM) using the SF of extra-Galactic radio sources (EGRS) and pulsars to find that the RM fluctuations we measure originate within the SNRs for all but the largest angular scales. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.02077.md
    + _build/html/tmp_2310.02077/./test_extraction_paper_workflow_alt.png
    + _build/html/tmp_2310.02077/./parameter_space_rchi2_paper.png
    + _build/html/tmp_2310.02077/./test_second_derivative.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\RomanNumeralCaps}[1]{\MakeUppercase{\romannumeral #1}}$
$\newcommand{\code}[1]{\texttt{#1}}$
$\newcommand{\saber}{{\small astro}\textsc{Saber}}$</div>



<div id="title">

# Cold atomic gas identified by $\ion{H}{i}$ self-absorption

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.02077-b31b1b.svg)](https://arxiv.org/abs/2310.02077)<mark>Appeared on: 2023-10-04</mark> -  _41 pages, 28 figures, accepted for publication in A&A_

</div>
<div id="authors">

<mark>J. Syed</mark>, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Stars form in the dense interiors of molecular clouds. The dynamics and physical properties of the atomic interstellar medium (ISM) set the conditions under which molecular clouds and eventually stars will form. It is, therefore, critical to investigate the relationship between the atomic and molecular gas phase to understand the global star formation process. Using the high angular resolution data from The $\ion{H}{i}$ /OH/Recombination line survey of the Milky Way (THOR), we aim to constrain the kinematic and physical properties of the cold atomic hydrogen gas phase toward the inner Galactic plane. $\ion{H}{i}$ self-absorption (HISA) has proven to be a viable method to detect cold atomic hydrogen clouds in the Galactic plane. With the help of a newly developed self-absorption extraction routine ( $\saber$ ), we build upon previous case studies to identify $\ion{H}{i}$ self-absorption toward a sample of Giant Molecular Filaments (GMFs). We find the cold atomic gas to be spatially correlated with the molecular gas on a global scale. The column densities of the cold atomic gas traced by HISA are usually of the order of $10^{20}\rm cm^{-2}$ whereas those of molecular hydrogen traced by $\element[][13]{CO}$ are at least an order of magnitude higher. The HISA column densities are attributed to a cold gas component that accounts for a fraction of $\sim$ 5 \% of the total atomic gas budget within the clouds. The HISA column density distributions show pronounced log-normal shapes that are broader than those traced by $\ion{H}{i}$ emission. The cold atomic gas is found to be moderately supersonic with Mach numbers of a $\sim$ few. In contrast, highly supersonic dynamics drive the molecular gas within most filaments. While $\ion{H}{i}$ self-absorption is likely to trace just a small fraction of the total cold neutral medium within a cloud, probing the cold atomic ISM by the means of self-absorption significantly improves our understanding of the dynamical and physical interaction between the atomic and molecular gas phase during cloud formation.

</div>

<div id="div_fig1">

<img src="tmp_2310.02077/./test_extraction_paper_workflow_alt.png" alt="Fig3" width="100%"/>

**Figure 3. -** Baseline extraction workflow of $\saber$ . In each panel, the black mock spectrum represents the observed $\ion${H}{i} emission spectrum, which is the sum of the three gray dashed components,  with self-absorption features (two red dashed components) superposed. The blue spectrum shows the "pure emission" spectrum that is to be recovered by the $\saber$  algorithm. The algorithm is then applied to the observed spectrum using the optimal smoothing parameters $(\lambda_1,\lambda_2)$. Hatched red areas indicate spectral channels that are masked out due to missing signal. _Left panel:_ The $\saber$  baseline (red) after the first major cycle iteration, that is, after the minor cycle smoothing converged given the input mock spectrum (i.e. after Eq. \eqref{equ:least_squ} has been solved for $\mathbf{z}$). _Middle panel:_ The $\saber$  baseline (red) after the last major cycle iteration, that is, after the major cycle smoothing converged and before adding the residual, which is the absolute difference between the first and last major cycle iteration. _Right panel:_ The final $\saber$  baseline (red) after adding the residual. The baseline so obtained reproduces the pure emission spectrum (blue) well. The resulting HISA features expressed as equivalent emission features are shown in green, and show a good match with the the real HISA absorption features. The smaller subpanels in each column show the residual, which is the difference between the red baseline and the blue emission spectrum, with the horizontal dotted red lines marking values of $\pm\sigma_\mathrm{rms}$. (*fig:mock_spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2310.02077/./parameter_space_rchi2_paper.png" alt="Fig1" width="100%"/>

**Figure 1. -** Smoothing parameter optimization using gradient descent. The map shows a sampled representation of the underlying $\vec\lambda$ parameter space in terms of the median value of the reduced chi square results. Initial values, tracks, and convergence locations of the $(\lambda_1,\lambda_2)$ parameters during the optimization are represented by black circles, black lines, and white crosses, respectively. The red cross marks the global minimum in the sampled parameter space. Initial locations that start off too far from the global best solution $(\lambda_1=3.5,\lambda_2=0.6)$ might converge to local minima with less accurate fit results. (*fig:parameter_space*)

</div>
<div id="div_fig3">

<img src="tmp_2310.02077/./test_second_derivative.png" alt="Fig2" width="100%"/>

**Figure 2. -** Second derivative representation as a means to identify self-absorption. _Top panel:_ The black mock spectrum represents the $\ion${H}{i} emission spectrum, with two self-absorption features superposed (red dashed components) and without any observational noise. The green spectrum shows the second derivative of the black mock spectrum, obtained from the finite differences between spectral channels. _Bottom panel:_ The black mock spectrum represents the $\ion${H}{i} emission spectrum, with two self-absorption features superposed (red dashed components) and with added noise that is comparable to the noise of the THOR-$\ion${H}{i} observations (same spectrum as in Fig. \ref{fig:mock_spectrum}). The green spectrum shows the second derivative of the black mock spectrum, obtained from the finite differences between spectral channels. The dashed blue spectrum represents a regularized least squares solution to the $\ion${H}{i} spectrum, which minimizes the second derivative. The corresponding second derivative is shown in blue, which is now less affected by noise fluctuations. (*fig:second_derivative*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.02077"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

286  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
